# Notebook 1: Data Exploration

This notebook explores the EXFOR experimental nuclear cross-section database
and AME2020 nuclear mass data. We examine:

1. Data coverage across the nuclear chart
2. Energy-dependent cross-section systematics
3. Distribution of measured cross-section magnitudes
4. Correlation between nuclear properties and cross-sections
5. Identification of data-sparse isotopic regions

These explorations motivate the feature engineering choices described in the methodology.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from pathlib import Path

# Project modules
from data.exfor_downloader import EXFORDownloader
from data.nuclear_properties import NuclearPropertiesLoader
from visualization.plots import plot_nuclear_chart_heatmap

plt.rcParams.update({
    'font.family': 'serif',
    'figure.dpi': 150,
})

print('Setup complete.')

## 1. Load EXFOR data

We load pre-downloaded EXFOR parquet files. If not yet downloaded, run:
```bash
python scripts/download_data.py --reactions n-gamma n-total n-elastic
```

In [ ]:
# Load EXFOR data for (n,gamma) reactions
exfor_path = Path('../data/raw/exfor/n_gamma_all.parquet')

if exfor_path.exists():
    exfor_df = pd.read_parquet(exfor_path)
    print(f'Loaded {len(exfor_df):,} (n,gamma) data points')
    print(f'Isotopes covered: {exfor_df[["Z","A"]].drop_duplicates().shape[0]}')
    print(f'Z range: {exfor_df["Z"].min()} - {exfor_df["Z"].max()}')
    print(f'Energy range: {exfor_df["energy_eV"].min():.2e} - {exfor_df["energy_eV"].max():.2e} eV')
    display(exfor_df.head())
else:
    print(f'EXFOR data not found at {exfor_path}.')
    print('Run: python scripts/download_data.py --reactions n-gamma')
    print('\nUsing synthetic demo data for notebook illustration...')
    # Create synthetic data for demonstration
    rng = np.random.default_rng(42)
    n_points = 5000
    Z_vals = rng.integers(1, 83, n_points)
    A_vals = Z_vals + rng.integers(0, Z_vals + 1)
    A_vals = np.clip(A_vals, Z_vals, Z_vals * 3)
    energies = 10 ** rng.uniform(-2, 7, n_points)
    sigma = 10 ** rng.uniform(-3, 5, n_points)
    exfor_df = pd.DataFrame({
        'Z': Z_vals, 'A': A_vals, 'N': A_vals - Z_vals,
        'energy_eV': energies, 'cross_section_barn': sigma,
        'reaction': 'N,G'
    })
    print(f'Generated {len(exfor_df):,} synthetic data points for demonstration.')

## 2. Data coverage on the nuclear chart

We visualize how many data points are available for each isotope (Z, N),
highlighting the large unmeasured regions.

In [ ]:
# Count data points per isotope
coverage = exfor_df.groupby(['Z', 'A']).size().reset_index(name='n_points')
coverage['N'] = coverage['A'] - coverage['Z']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Number of data points
scatter = axes[0].scatter(
    coverage['N'], coverage['Z'],
    c=np.log10(coverage['n_points']),
    cmap='YlOrRd', s=8, marker='s',
    vmin=0, vmax=np.log10(coverage['n_points'].max())
)
cbar = plt.colorbar(scatter, ax=axes[0], fraction=0.046, pad=0.04)
cbar.set_label('log$_{10}$(number of data points)')

# Magic number lines
for m in [2, 8, 20, 28, 50, 82, 126]:
    axes[0].axvline(m, color='gray', lw=0.4, alpha=0.7, ls=':')
    axes[0].axhline(m, color='gray', lw=0.4, alpha=0.7, ls=':')

axes[0].set_xlabel('Neutron number N')
axes[0].set_ylabel('Proton number Z')
axes[0].set_title('EXFOR (n,$\\gamma$) data coverage')

# Panel 2: Histogram of data points per isotope
axes[1].hist(
    np.log10(coverage['n_points']),
    bins=30, color='steelblue', edgecolor='none', alpha=0.8
)
axes[1].set_xlabel('log$_{10}$(data points per isotope)')
axes[1].set_ylabel('Number of isotopes')
axes[1].set_title('Distribution of data density')
axes[1].axvline(np.log10(coverage['n_points'].median()), 
                color='red', ls='--', label=f'Median: {coverage["n_points"].median():.0f}')
axes[1].legend()

plt.tight_layout()
plt.savefig('../results/figures/exfor_coverage_chart.pdf', bbox_inches='tight', dpi=300)
plt.show()
print(f'Total isotopes with data: {len(coverage)}')
print(f'Median data points per isotope: {coverage["n_points"].median():.0f}')
print(f'Isotopes with fewer than 10 points: {(coverage["n_points"] < 10).sum()}')

## 3. Energy-dependent cross-section systematics

Cross-sections vary by many orders of magnitude with incident neutron energy.
We examine this energy dependence across different mass regions.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

mass_regions = {
    'Light (A < 40)': exfor_df[exfor_df['A'] < 40],
    'Medium (40-100)': exfor_df[(exfor_df['A'] >= 40) & (exfor_df['A'] <= 100)],
    'Heavy (100-209)': exfor_df[(exfor_df['A'] > 100) & (exfor_df['A'] <= 209)],
    'Actinides (A>209)': exfor_df[exfor_df['A'] > 209],
}

for ax, (label, subset) in zip(axes.flat, mass_regions.items()):
    if len(subset) == 0:
        ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(label)
        continue
    
    ax.scatter(
        subset['energy_eV'], subset['cross_section_barn'],
        s=1, alpha=0.2, color='steelblue', rasterized=True
    )
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Neutron energy (eV)')
    ax.set_ylabel('Cross-section (barn)')
    ax.set_title(f'{label} (n={len(subset):,} points)')
    ax.grid(True, which='both', alpha=0.3, lw=0.5)

plt.suptitle('(n,$\\gamma$) cross-sections by mass region', y=1.01)
plt.tight_layout()
plt.savefig('../results/figures/cross_section_vs_energy_by_region.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 4. Cross-section magnitude distribution

In [ ]:
thermal_mask = (exfor_df['energy_eV'] >= 0.02) & (exfor_df['energy_eV'] <= 0.05)
thermal_df = exfor_df[thermal_mask].copy()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Distribution of thermal cross-sections
if len(thermal_df) > 10:
    log_sigma = np.log10(thermal_df['cross_section_barn'].clip(lower=1e-6))
    axes[0].hist(log_sigma, bins=40, color='steelblue', edgecolor='none', alpha=0.8)
    axes[0].set_xlabel('log$_{10}$($\\sigma_{thermal}$ / barn)')
    axes[0].set_ylabel('Count')
    axes[0].set_title(f'Thermal (n,$\\gamma$) cross-section distribution (n={len(thermal_df):,})')
    axes[0].axvline(log_sigma.median(), color='red', ls='--', 
                   label=f'Median: {10**log_sigma.median():.1f} barn')
    axes[0].legend()
else:
    log_sigma = np.log10(exfor_df['cross_section_barn'].clip(lower=1e-6))
    axes[0].hist(log_sigma, bins=40, color='steelblue', edgecolor='none', alpha=0.8)
    axes[0].set_xlabel('log$_{10}$($\\sigma$ / barn)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('All-energy (n,$\\gamma$) cross-section distribution')

# Cross-section vs mass number
if len(thermal_df) > 10:
    scatter_df = thermal_df.groupby('A')['cross_section_barn'].median().reset_index()
else:
    scatter_df = exfor_df.groupby('A')['cross_section_barn'].median().reset_index()

axes[1].semilogy(scatter_df['A'], scatter_df['cross_section_barn'], 
                 'o', ms=4, alpha=0.6, color='darkblue')
axes[1].set_xlabel('Mass number A')
axes[1].set_ylabel('Median cross-section (barn)')
axes[1].set_title('Median cross-section vs mass number')
axes[1].grid(True, which='both', alpha=0.3, lw=0.5)

plt.tight_layout()
plt.savefig('../results/figures/cross_section_distribution.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 5. Load nuclear properties and examine correlations

We examine the Pearson and Spearman correlations between nuclear structure
features and measured cross-sections to validate the feature engineering choices.

In [ ]:
props_loader = NuclearPropertiesLoader(
    ame_path='../data/external/ame2020_mass.txt',
    nubase_path='../data/external/nubase2020.txt',
)

# Check if AME data is available
ame_path = Path('../data/external/ame2020_mass.txt')
if ame_path.exists():
    props_df = props_loader.load_all()
    print(f'Loaded nuclear properties for {len(props_df)} isotopes')
    
    # Merge with EXFOR thermal data
    merged = exfor_df.merge(props_df, on=['Z', 'A'], how='inner')
    
    feature_cols = [
        'binding_energy_per_A_MeV', 'Sn_MeV', 'S2n_MeV',
        'isospin_asym', 'dist_to_magic_Z', 'dist_to_magic_N'
    ]
    feature_cols = [c for c in feature_cols if c in merged.columns]
    
    merged['log10_sigma'] = np.log10(merged['cross_section_barn'].clip(lower=1e-10))
    
    # Correlation heatmap
    corr_cols = feature_cols + ['log10_sigma']
    available = [c for c in corr_cols if c in merged.columns]
    corr = merged[available].corr(method='spearman')
    
    fig, ax = plt.subplots(figsize=(8, 7))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=ax, annot_kws={'size': 9})
    ax.set_title('Spearman correlation: nuclear features vs log(cross-section)')
    plt.tight_layout()
    plt.savefig('../results/figures/feature_correlation_heatmap.pdf', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print('AME2020 data not available. Skipping correlation analysis.')
    print('Run: python scripts/download_data.py --download-nuclear-props')

## 6. Summary statistics

In [ ]:
print('=== EXFOR Data Summary ===')
print(f'Total data points: {len(exfor_df):,}')
print(f'Unique isotopes: {exfor_df[["Z","A"]].drop_duplicates().shape[0]}')
print(f'Z range: {exfor_df["Z"].min()} - {exfor_df["Z"].max()}')
print(f'A range: {exfor_df["A"].min()} - {exfor_df["A"].max()}')
print(f'Energy range: {exfor_df["energy_eV"].min():.2e} - {exfor_df["energy_eV"].max():.2e} eV')
print(f'\nCross-section range: {exfor_df["cross_section_barn"].min():.2e} - {exfor_df["cross_section_barn"].max():.2e} barn')
print(f'log10 range: {np.log10(exfor_df["cross_section_barn"].min()):.1f} to {np.log10(exfor_df["cross_section_barn"].max()):.1f}')
print(f'\nIsotopes with >100 data points: {(exfor_df.groupby(["Z","A"]).size() > 100).sum()}')
print(f'Isotopes with <5 data points:   {(exfor_df.groupby(["Z","A"]).size() < 5).sum()}')